# Hands On Astropy Tutorial

This hands-on tutorial gives an introduction and overview of the [Astropy](http://www.astropy.org/) Python package for astronomy.


![](astropy_banner_96.png)


## Preface

We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. Every sub-topic we will cover in this tutorial will be concluded by a few **exercises with different levels of difficulty** (*easy*, *advanced* and *hard*). We will give you **~10-15 min to solve the exercises** and present a possible solution afterwards. In case we don't have time to show all the solutions or in the likely case that you don't manage to solve all of them: we will provide a **sample solution** for all exercises **after the course**.   

The estimated time for this tutorial is ~2 hours. Feel free to **interrupt at any time** to ask questions or **talk to the assisting tutors** when you encounter errors or the code doesn't work as expected.



## Overview

The material presented here is partly based on: 

- Notebooks provided in the [Astropy Tutorials](https://github.com/astropy/astropy-tutorials) repository.
- And the [Astropy Introduction for Gammapy Users](https://github.com/gammapy/gammapy-extra/blob/master/notebooks/astropy_introduction.ipynb) notebook.


The Astropy package is structured into several submodules and we will cover (what we consider) the most important of them in the following order:

- [astropy.units](http://docs.astropy.org/en/stable/units/index.html) and in particular [astropy.units.Quantities](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html) to do astronomical calculations with units.

- [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) and in particular the classes [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) and [Angle](http://docs.astropy.org/en/stable/coordinates/angles.html) to handle astronomical sky positions, coordinate systems and coordinate transformations.

- [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) to open and write data files in [FITS format](https://fits.gsfc.nasa.gov/fits_documentation.html).

- [astropy.tables](http://docs.astropy.org/en/stable/table/index.html) and [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) class to handle astronomical data tables.

- (optional) Plotting os astronomical sky images with [astropy.visualization.wcsaxes](http://docs.astropy.org/en/stable/visualization/wcsaxes/)
#- (optional) Define regions in the sky with the [region](http://astropy-regions.readthedocs.io/en/latest/getting_started.html) package
- (optional) manipulate [Times and Dates](http://docs.astropy.org/en/stable/time/index.html)



## What is Astropy?


    "The Astropy Project is a community effort to develop a single core package for Astronomy in Python and foster              interoperability between Python astronomy packages."


[Astropy paper](http://adsabs.harvard.edu/abs/2013A%26A...558A..33A)

In addition to the Astropy core package there is an infrastructure of  [Astropy affiliated packages](http://www.astropy.org/affiliated/) dedicated to specific fields od astronomy or analysis tasks:




Let's start with the setup and check of the notebook:

# Setup

In [1]:
# to make plots appear inline in the notebook
%matplotlib inline  
import matplotlib.pyplot as plt

In [2]:
# If something below doesn't work, here's how you
# can check what version of Numpy and Astropy you have
# All examples should work with Astropy > 1.3 and Numpy > 1.11
import numpy as np
import astropy
print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

numpy: 1.11.3
astropy: 1.3


# Quantities

In [3]:
from astropy import units as u

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

## See also
See also this [tutorial](http://www.astropy.org/astropy-tutorials/Quantities.html)

# Coordinates

In [4]:
from astropy.coordinates import SkyCoord

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Fits I/O

The flexible image transport system format (FITS) is widely used data format for astronomical images and tables.

https://fits.gsfc.nasa.gov/fits_documentation.html

In [5]:
from astropy.io import fits

# Exercises

- (*easy*)
- (*advanced*)
- (*hard*)

# Tables

In [6]:
from astropy.table import Table

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Plotting

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Acknowledgements

This tutorial was supported by the H2020-Astronomy ESFRI and Research Infrastructure Cluster (Grant Agreement number: 653477).